In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import pickle

In [2]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Audf

In [3]:
with open('../input/entities/entityAu.pkl', 'rb') as f:
    entitiesA = pickle.load(f)

In [4]:
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

Audf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})



for i in entitiesA:
    
 dc={}
 P=[]
 D=[]
 G=[]
 O=[]
 for entity in i:
    if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        
        
        
 for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
 Per.append(P)
 Date.append(D)
 ORG.append(O)
 GPE.append(G)
            
Audf["Date-ontonotes"]=Date
Audf["PERSON-ontonotes"]=Per
Audf["GPE-ontonotes"]=GPE
Audf["ORG-ontonotes"]=ORG    

In [5]:
Audf

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,NaN,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]"
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864]
2,NaN,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]"
3,NaN,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]"
4,NaN,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]"
...,...,...,...,...,...
2131,NaN,"[Geo . Little, Geo . B. Ayer]","[Raleigh, N.C., Wake County, N.C, United State...",[],"[May 26, July 14]"
2132,NaN,[Birnie],[],[],[July 30th 1867]
2133,NaN,[William Birnie],"[North Carolina, Robeson County, United States]",[],[30 day of July]
2134,NaN,"[A.L . Price, James Fulton, John D Conoley, Al...","[North Carolina, New Hanover County, New Hanov...",[Fulton Price],"[May 29 1865, 15th Sept 1865, 15th September A..."


In [6]:
G=[]
for i in Audf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
Audf["postDate-ontonotes"]=GF

In [7]:
G=[]
for i in Audf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

Audf["postDate-ontonotes"]=G

In [8]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta","Aug."]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh',"Sep."]:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The","Dec"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta","Aug."]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This ","Sept.","Sep."]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber","Dec."]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

**removing duplicated dates**

In [9]:
def uniqueDate(ite):
    seen = set()
    result = []
    for item in ite:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
Nd=[]
for i in NG:
    i=uniqueDate(i)
    Nd.append(i)

**removing the date of the issue of low**

In [10]:
for i in Nd:
    for j in i:
        if j=="29-May-1865":
            i.remove(j)

1. if there is 2 dates : the first one almost is the right one 
1. if there is more than  2 dates the second one almost the right one

In [11]:
for i in range(len(Nd)):
    if len(Nd[i])==2:
        Nd[i].remove(Nd[i][1])
    elif (len(Nd[i])>2):
        v=Nd[i][1]
        del(Nd[i][0:])
        Nd[i]=[v]

In [12]:
Audf["Dates"]=Nd

In [13]:
Audf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates
0,NaN,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865]
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[]
2,NaN,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865]
3,NaN,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[]
4,NaN,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867]


In [14]:
Audf["text"]=range(2136)

# dflarge

In [15]:
with open('../input/entities/entityL.pkl', 'rb') as f:
    entitiesL= pickle.load(f)

In [16]:
len(entitiesL)

2136

In [17]:
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in entitiesL:
    dc={}
    P=[]
    L=[]
    G=[]
    O=[]
    for entity in i :

                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                       
        
      
    for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
    Per.append(P)
    ORG.append(O)
    LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG   

In [18]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,NaN,"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,NaN,[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,NaN,"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,NaN,[God],"[State of, North Carolina, United States]",[Constitution]
4,NaN,"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"


# final df

In [19]:
Audf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
Audf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
Audf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]

In [20]:
Audf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge
0,0,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,1,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[],[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,2,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865],"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,3,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[],[God],"[State of, North Carolina, United States]",[Constitution]
4,4,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867],"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"


In [21]:
import os 
os.getcwd()

'/kaggle/working'

In [22]:
Audf.to_pickle("data.pkl")

In [23]:
with open('../input/audfff/data.pkl', 'rb') as f:
    data= pickle.load(f)

In [24]:
data

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,postDate-ontonotes,Dates,Person-NerEngLarge,LOC-NerEngLarge,ORG-NerEngLarge
0,0,"[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]",[16-Sept-1865],[16-Septembre-1865],"[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,1,[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864],[],[],[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,2,"[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]",[3-October-1865],[3-October-1865],"[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,3,[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]",[],[],[God],"[State of, North Carolina, United States]",[Constitution]
4,4,[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]",[31-May-1867],[31-May-1867],"[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"
...,...,...,...,...,...,...,...,...,...,...
2131,2131,"[Geo . Little, Geo . B. Ayer]","[Raleigh, N.C., Wake County, N.C, United State...",[],"[May 26, July 14]",[],[],"[Geo . Little, Geo, B. Ayer, Hugo Hillebrandt]","[Raleigh, N.C., Wake County, State of N.C, Uni...",[]
2132,2132,[Birnie],[],[],[July 30th 1867],[30-July-1867],[30-July-1867],[],[],[]
2133,2133,[William Birnie],"[North Carolina, Robeson County, United States]",[],[30 day of July],[],[],"[William Birnie, God, Wm Birnie, W.A]","[State of, North Carolina, Robeson County, Uni...",[Constitution]
2134,2134,"[A.L . Price, James Fulton, John D Conoley, Al...","[North Carolina, New Hanover County, New Hanov...",[Fulton Price],"[May 29 1865, 15th Sept 1865, 15th September A...","[29-May-1865, 15-Sept-1865, 15-September-1865,...","[29-May-1865, 15-Septembre-1865, 15-Septembre-...","[Price, James Fulton, Sgd W . N. Peden, Miles ...","[State of North Carolina, New Hanover County, ...","[Fulton Price, Confederate Government, Court o..."


# Separate Oath maker and witness name

In [25]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

In [26]:
all_text=newdata["transcription_text"]
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punctuation
punct='!"#$%&\'()*+-/<=>?@[\\]^`{|}~'

In [27]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] 
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:00<00:00, 2935.83it/s]


In [28]:
allPerson=[]
for i in Audf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in Audf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in Audf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in Audf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in Audf["ORG-ontonotes"]:
            alORGl.append(i)

In [29]:
Personlarge=[]
for i in dflarge["PERSON-nerenglarge"]:
            Personlarge.append(i)
ORGlarge=[]
for i in dflarge["ORG-nerenglarge"]:
            ORGlarge.append(i) 
LOClarge=[]
for i in dflarge["LOC-nerenglarge"]:
            LOClarge.append(i)

In [30]:
# NewPerson[10]

In [31]:
# Perr[10]

In [32]:
# NewPerson[25]

In [33]:
# Perr[25]

In [34]:
# NewPerson[112]

In [35]:
# Perr[112]

In [36]:
# NewPerson[211]

In [37]:
# Perr[211]

In [38]:
# NewPerson[218]

In [39]:
# Perr[218]

In [40]:
NewPerson=[]
for i in Personlarge:
    p=[]
    for j in i:
        j=j.replace('.','')
        j=j.replace('Oath','')
        j=j.replace('Sworn','')
        j=j.replace('Path of','')
        j=j.replace('  ',' ')
        p.append(j)
    NewPerson.append(p)
    
        

In [41]:
len(NewPerson)

2136

In [42]:
PL=[]
for i in NewPerson:
    for j in i:
        PL.append(j)
len(PL)

9512

# removing outliers

In [43]:
L=["God","GOD","god","ANDREW JOHNSON","AB","CE","andrew jonhson","Justice of Peace","Almighty God","Andrew Johnson","SE","TD","Jm","I","I J","MW","M","JH","JB","WD","JK","J P","C","G","HA","J","A","F A","DS","CT","RF",'JE',"Ashe","JL","H","JP","Mr","Sworn","sworn","JF","Bonnell","B","ER","WHé","JH","JW","F","Wm","W","WF"]

In [44]:
Per=[]
for i in NewPerson:
    p=[]
    for j in i:
        if j=="Almighty":
            j="Almighty God"
        if j not in L:
            p.append(j)
        
    Per.append(p)

In [45]:
all_text[1006]

'United States of America I , Elizabeth W. Bonnell of Barnwell District State of South Carolina , solemnly swear affirm presence of Almighty God , I henceforth support defend Consitution of United States , service of States thereunder , I like manner abide support law proclamation made existing rebellion reference emancipation of slave help God . Elizabeth W Bonnell Subscribed sworn Blackville 21st day of October A.D. 1865 James E. Robinson Magst .'

In [46]:
Per[1006]

['Elizabeth W Bonnell', 'Elizabeth W Bonnell', 'James E Robinson Magst']

In [47]:
# Perr[1006]

In [48]:
PLl=[]
for i in Per:
    for j in i:
        PLl.append(j)
len(PLl)

5683

# removing redundant name

In [49]:
def uniqueName(iterable):
    seen = set()
    result = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result
Perr=[]
for i in Per:
    i=uniqueName(i)
    Perr.append(i)
    

In [50]:
PL=[]
for i in Perr:
    for j in i:
        PL.append(j)
len(PL)

5318

In [51]:
P=[]
for i in Perr:
    L=[]
    for j in range(len(i)):
        L.append(i[j].split())
    P.append(L)


In [52]:
Perr[1000:1050]

[['John Bowers', 'John E Bowers', 'Levi Stuber'],
 ['J Cheesborough', 'Levi Stuber'],
 ['Wm H Gilliland', 'Levi Stuber'],
 ['WA Wardlaw', 'J Miller Thompson', 'Daniel Lesesne', 'B Eaton'],
 ['Levi Stuber', 'Thomas Bonnell'],
 ['John Bonnell', 'James E Robinson Magst'],
 ['Elizabeth W Bonnell', 'James E Robinson Magst'],
 ['Caroline B Bonnell', 'James E Robinson Magstr'],
 ['Wm S Lanneau', 'Wm S Lamreau', 'Levi Stuber'],
 ['Thomas Bonnell', 'Levi Stuber'],
 ['Mary J Lockwood', 'Laurens', 'J E Ashe', 'GOD '],
 ['Caroline L Lockwood', 'JE Ashe'],
 ['Jane B Lockwood', 'J W Lockwood', 'Laurens', 'JW Lockwood'],
 ['Lockwood', 'Levi Stuber', 'I J Lockwood', 'J Lockwood'],
 ['James Bancroft', 'Levi Stuber', 'Jas Bancroft'],
 ['Walter Blake', 'Johnsons', 'BF', 'Perry'],
 ['Adeline E Brisbane', 'A E Brisbane', 'O', 'Shepherd', 'GOD ', 'O Shepherd'],
 ['Regina M Baker', 'Levi Stuber'],
 ['Esther Lynch Bowman',
  'EL Bowman',
  'Sumter',
  'So Carol',
  'TB Johnston',
  'E L Bowman'],
 ['James E T

In [53]:
Perr[0]

['I C Meekins', 'Isaac C Meekins', 'E H Walker']

In [54]:
P[1000]

[['John', 'Bowers'], ['John', 'E', 'Bowers'], ['Levi', 'Stuber']]

In [55]:
P[0]

[['I', 'C', 'Meekins'], ['Isaac', 'C', 'Meekins'], ['E', 'H', 'Walker']]

In [56]:
P[50]

[['Albert', 'C', 'Beals'], ['Jas', 'C', 'Davis']]

In [57]:
P[1000]

[['John', 'Bowers'], ['John', 'E', 'Bowers'], ['Levi', 'Stuber']]

In [58]:
l=[[['I', 'C', 'Meekins'], ['Isaac', 'C', 'Meekins'], ['E', 'H', 'Walker']],[['John', 'Bowers'], ['John', 'E', 'Bowers'], ['Levi', 'Stuber']]]

In [59]:
# fiPer=[]
# for i in l:
   
#     L=[]
#     for j in range(len(i)-1):
#         if ((i[j][0]==i[j+1][0]) & (i[j][len(i[j])]==i[j+1][len(i[j])])) |(i[j][0]==i[j+1][0]):
#                 continue
#         L.append(i[j])
#     fiPer.append(L)

In [60]:
# fiPer

In [61]:
# fiPer=[]
# for i in l:
   
#     L=[]
#     for j in range(len(i)):
#         if ((i[j][0]==i[j+1][0]) & (i[j][len(i[j])-1]==i[j+1][len(i[j])-1])) | (i[j][0]==i[j+1][0]):
#                  print(i[j])

        
                
                
            
        
        

##### 